In [45]:
import os
import time
import json
import pandas as pd
import torch
import loader
import utils
import preprocess_data
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_bi_daf
from config import config_ensemble
from modules import match_lstm
from modules import r_net
from modules import bi_daf
from torch.nn import functional as f

# config = config_match_lstm.config
# config = config_r_net.config
config = config_bi_daf.config
# config = config_ensemble.config

In [2]:
lang = loader.load_vocab(config.vocab_path)

In [3]:
embedding_np = loader.load_w2v(config.embedding_path)

In [4]:
test_data = loader.load_data(config.test_df, lang)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.448 seconds.
Prefix dict has been built succesfully.


In [36]:
test_loader = loader.build_loader(
        dataset=test_data[:2],
        batch_size=5,
        shuffle=False,
        drop_last=False
    )

In [37]:
param = {
        'embedding': embedding_np,
        'embedding_type': config.embedding_type,
        'embedding_is_training': config.embedding_is_training,
        'mode': config.mode,
        'hidden_size': config.hidden_size,
        'dropout_p': config.dropout_p,
        'encoder_dropout_p': config.encoder_dropout_p,
        'encoder_bidirectional': config.encoder_bidirectional,
        'encoder_layer_num': config.encoder_layer_num,
        'is_bn': config.is_bn
    }

In [38]:
model = eval(config.model_name).Model(param)
model = model.cuda()
model.train()
model_path = os.path.join('model', config.model_save)
print('load model, ', model_path)
state = torch.load(model_path)
model.load_state_dict(state['best_model_state'])

load model,  model/bi_daf_1


In [39]:
batch_0 = 0
for i, batch in enumerate(test_loader):
    if i ==0:
        batch_0 = batch
    break

In [40]:
batch_0= utils.deal_batch(batch_0)

In [41]:
outputs = model(batch_0)

In [43]:
a = torch.Tensor([-float('inf'), -float('inf')])

In [46]:
f.softmax(a)

/home/xy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor([nan., nan.])